In [1]:
import h5py
import scanpy as sc
import anndata
import loompy as lp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import glob
import importlib

In [2]:
import sys
sys.path.insert(0, '/home/tchari/monod/src/')

In [3]:
import monod
importlib.reload(monod)
from monod import preprocess, extract_data, cme_toolbox, analysis, mminference

In [5]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

In [6]:
import h5py
import scanpy as sc
import anndata
import loompy as lp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import glob
import importlib

import sys
sys.path.insert(0, '/home/tchari/monod/src/')

import monod
importlib.reload(monod)
from monod import preprocess, extract_data, cme_toolbox, analysis, mminference

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

## **Checking meK-Means performance with simulated dropout**

In [17]:
d_strings  = glob.glob('/home/tchari/counts/meKMeans_sims/loom/*.loom')
d_strings 

['/home/tchari/counts/meKMeans_sims/loom/drop_1perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/drop_5perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/drop_20perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/negctrl_sim_k1.loom',
 '/home/tchari/counts/meKMeans_sims/loom/b_beta_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/k_gamma_sim_k8.loom']

In [18]:
attribute_names =[('unspliced','spliced'),'gene_name','barcode']

In [19]:
datas = [i.replace('/home/tchari/counts/meKMeans_sims/loom/','') for i in d_strings]
datas = [i.replace('.loom','') for i in datas]

In [20]:
to_run = [0,1,2,4] #inds to run

In [21]:
samp = [-6.777777777777778, -1.25]

In [22]:
transcriptomes = '/home/tchari/perturbCME/notebooks/sim_genome_polyA_cum_3_0'

In [53]:
sub_d_strings = [d_strings[i] for i in to_run]
sub_d_strings

['/home/tchari/counts/meKMeans_sims/loom/drop_1perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/drop_5perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/drop_20perc_sim_k8.loom',
 '/home/tchari/counts/meKMeans_sims/loom/b_beta_sim_k8.loom']

In [56]:
datas_sub = [datas[i] for i in to_run]
datas_sub

['drop_1perc_sim_k8',
 'drop_5perc_sim_k8',
 'drop_20perc_sim_k8',
 'b_beta_sim_k8']

In [ ]:
#For each loom in looms from ./hvg_objs

# !mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0209
!mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0309


dir_strings = []
data_strings = []
result_strings = []

#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
meK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in sub_d_strings: #************* FASTER FOR NOW ************* 
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas_sub]
    
    #Inference params for sampling
    tech_tup = samp
    
    #Preprocessing params for filepaths
    trans = transcriptomes

    data_name = np.array(datas_sub)[ind][0]
    
    
    ds = lp.connect(d)
    g_names = ds.ra[attribute_names[1]]
    num_genes = len(g_names)
    num_cells = len(ds.ca[attribute_names[2]])
    
    U = ds.layers['unspliced'][:,:].T
    S = ds.layers['spliced'][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()

    
    print(data_name+' with '+str(num_genes)+' genes')
    
    
   
    Ks = [5,len(np.unique(true_labs)),len(np.unique(true_labs))+5,len(np.unique(true_labs))+10]
        

    for clus in Ks:
       
        #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
        dir_string,dataset_string = monod.preprocess.construct_batch([d], \
                                                     trans, \
                                                     [data_name], \
                                                     attribute_names=attribute_names,\
                                                     batch_location='./fits/meKruns_0309', \
                                                     meta=data_name+'_K'+str(clus), \
                                                     batch_id=1, \
                                                     n_genes=num_genes,exp_filter_threshold=None,
                                                                     viz=False,genes_to_fit=g_names)
                                                        #num_genes-60 or 100,genes_to_fit=markers
        dir_strings += [dir_string]
        data_strings += [dataset_string]

        #Set model and run meK-Means
        
        #Define bounds for params
        phys_lb = [-3.0, -3, -3 ] 
        phys_ub = [4.2, 2.5, 2.5] 
        samp_lb = tech_tup 
        samp_ub = tech_tup  
        gridsize = [1,1] #Already have tech params
    

        epochs = 10

        # ---------------- meK-Means Inference ---------------- 
        #Define model with bursty transcription and Poisson molecule capture/sampling
        fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

        #Set up mminference parameters
        inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                    dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
                    gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

        #Read in loom file with filtered barcodes
        search_data = monod.extract_data.extract_data(d, trans, data_name,
                    dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

        #Run inference(fit_all_grid_points()) and Save result file strings
        full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

        result_strings.append(full_result_string)
        

        # ----------------  Save output ---------------- 
        #Read in results and get cluster assignments
        sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
        cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
        for i in range(len(sr)):
            r = sr[i]
            cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

        print('No. clus: ',len(np.unique(cat_assigns)))
        

        meths += ['meK-Means']
        result += [cat_assigns]
        mats += ['U,S']
        hyper += [clus]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,cat_assigns)]
        ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]
        
        print('ARI: ',adjusted_rand_score(true_labs,cat_assigns))


meK_res['Method'] = meths
meK_res['Clustering'] = result
meK_res['Matrices'] = mats
meK_res['Hyperparam'] = hyper
meK_res['Genes'] = hvgs
meK_res['Data'] = data
meK_res['ARI'] = ari
meK_res['AMI'] = ami
    

mkdir: cannot create directory ‘/home/tchari/perturbCME/notebooks/fits/meKruns_0309’: File exists
drop_1perc_sim_k8 with 1100 genes


  0%|                                                                                                              | 0/5 [00:00<?, ?it/s]